In [1]:
#@title Library and Environment
import pandas as pd
import fmpsdk
import os
import dotenv
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from fmp import FMP
from PyPDF2 import PdfMerger

dotenv.load_dotenv()
apikey:str=os.getenv("apikey")
stock_symbol: str = input("Please enter stock symbol: ")

report_path:str = os.path.join("report_output",f"{stock_symbol}_finReport.pdf")
# util function
def to_percentage(num:float)->str:
  return f'{num*100:.2f} %'

def remove_prevReport(path:str):
  
  if os.path.isfile(path):
    os.remove(path)
    print(f"previous {path} is removed")
    return
  
  print(f"the directory is clean, no previous version exists")

def save_fig(path:str) -> None:
  merge_temp(path,"temp_append.pdf")

def save_dataframe(df:pd.DataFrame,title:str,  path:str) -> None:

  # table formatting
  transformed_frame:pd.DataFrame = df.map(lambda x: round(x,2) if isinstance(x,float) else x)
  tabFig , tabAx = plt.subplots()
  tabAx.axis('tight')
  tabAx.axis('off')
  
  table1 = tabAx.table(cellText=transformed_frame.values,
                        colLabels=transformed_frame.columns,
                        rowLabels=transformed_frame.index,
                        cellLoc = 'center',
                        loc='center')
  tabFig.suptitle(title)
  # 
  merge_temp(path,"temp_append.pdf")

def merge_temp(path:str,temp_pdf:str) -> None:
    with PdfPages(temp_pdf) as pdf:
      pdf.savefig(bbox_inches="tight")
      plt.close()
      print("figure is saved")
    merger:PdfMerger= PdfMerger()
    if os.path.exists(path):
      merger.append(path)
    merger.append(temp_pdf)
    merger.write(path)
    merger.close()
    os.remove(temp_pdf)


remove_prevReport(report_path)
print(f"Fetching financial data for {stock_symbol}...")

the directory is clean, no previous version exists
Fetching financial data for 原相(3227)...


In [2]:
#@title JSON data
import json
import os

balance_statements:list[dict] = []
income_statements:list[dict] = []
cashflow_statements:list[dict] = []

report_directory:str = "original_reports"

print(f"report directory: {report_directory}")

with open(os.path.join(report_directory,"balance_statements"
".json"), 'r') as f:
    balance_data:dict = json.load(f)
    balance_statements = balance_data["balance_sheets"]
with open(os.path.join(report_directory,"income_statements.json"), 'r') as f:
    income_data:dict = json.load(f)
    income_statements = income_data["income_statements"]
with open(os.path.join(report_directory,"cashflow_statements.json"), 'r') as f:
    cashflow_data:dict = json.load(f)
    cashflow_statements = cashflow_data["cashflow_statements"]


report directory: original_reports


In [3]:
balance_frames:pd.DataFrame = pd.DataFrame(balance_statements).set_index('date').sort_index()
income_frames:pd.DataFrame = pd.DataFrame(income_statements).set_index('date').sort_index()
cashflow_frames:pd.DataFrame = pd.DataFrame(cashflow_statements).set_index('date').sort_index()
balance_frames

,total_asset,total_liability,totalDebt,totalEquity,current_assets,current_liabilities,shareholderEquity,inventory,account_receivable,account_payable
date,,,,,,,,,,
2020,11880621000,2943135000,436553000,8937486000,7387895000,2381035000,8937486000,769456000,1491508000,1908538000
2021,13518620000,3322800000,447068000,10195820000,7863602000,2806521000,10195820000,1046335000,1015372000,2326171000
2022,11283652000,2150857000,22800000,9132795000,6239764000,1661447000,9132795000,988022000,637041000,1605318000
2023,12175855000,2272963000,21884000,9902892000,6718669000,1841632000,9902892000,680462000,992273000,1620576000
2024,14240437000,2681811000,35049000,11558626000,8191824000,2224356000,11558626000,950771000,1135362000,2095225000


In [4]:
income_frames

,revenue,cost_of_goods_sold,gross_profit,operating_expenses,operating_income,net_income,eps,interest_expense,share_outstanding
date,,,,,,,,,
2020,8148017000,3429657000,4718360000,946378000,1849005000,1456606000,10.60,-5568000,137415660
2021,8800642000,3778000000,5022642000,1027567000,1824801000,1575935000,11.32,-7093000,139216873
2022,5225907000,2405685000,2820222000,903388000,8920000,1050768000,7.41,-4800000,141804049
2023,5844521000,2478220000,3366301000,857260000,714102000,799400000,5.57,-3757000,143518851
2024,8362273000,3175229000,5187044000,1037499000,1758657000,1815418000,12.52,-4723000,145001438
TTM,9143383000,3487908000,5655475000,978732000,2011944000,1911672000,13.10,-5149000,145993280


In [5]:
cashflow_frames

,operating_cash_flow,investing_cash_flow,free_cashflow
date,,,
2020,1656987000,-1789706000,-1272727000
2021,2763452000,-1175275000,1588177000
2022,295403000,332677000,62808000
2023,1290750000,-195038000,1095712000
2024,2509485000,-1029228000,1470257000
TTM,2767672000,-226113000,2544565000


In [6]:
combine_frames:pd.DataFrame = pd.concat([balance_frames, income_frames, cashflow_frames], axis=1,join='inner')
combine_frames

,total_asset,total_liability,totalDebt,totalEquity,current_assets,current_liabilities,shareholderEquity,inventory,account_receivable,account_payable,...,gross_profit,operating_expenses,operating_income,net_income,eps,interest_expense,share_outstanding,operating_cash_flow,investing_cash_flow,free_cashflow
date,,,,,,,,,,,,,,,,,,,,,
2020,11880621000,2943135000,436553000,8937486000,7387895000,2381035000,8937486000,769456000,1491508000,1908538000,...,4718360000,946378000,1849005000,1456606000,10.60,-5568000,137415660,1656987000,-1789706000,-1272727000
2021,13518620000,3322800000,447068000,10195820000,7863602000,2806521000,10195820000,1046335000,1015372000,2326171000,...,5022642000,1027567000,1824801000,1575935000,11.32,-7093000,139216873,2763452000,-1175275000,1588177000
2022,11283652000,2150857000,22800000,9132795000,6239764000,1661447000,9132795000,988022000,637041000,1605318000,...,2820222000,903388000,8920000,1050768000,7.41,-4800000,141804049,295403000,332677000,62808000
2023,12175855000,2272963000,21884000,9902892000,6718669000,1841632000,9902892000,680462000,992273000,1620576000,...,3366301000,857260000,714102000,799400000,5.57,-3757000,143518851,1290750000,-195038000,1095712000
2024,14240437000,2681811000,35049000,11558626000,8191824000,2224356000,11558626000,950771000,1135362000,2095225000,...,5187044000,1037499000,1758657000,1815418000,12.52,-4723000,145001438,2509485000,-1029228000,1470257000


In [7]:
combine_frames["debtToEquity"] = round(combine_frames["totalDebt"] / combine_frames["totalEquity"],2)
combine_frames["returnOnEquity"] = round(combine_frames["net_income"] / combine_frames["totalEquity"],2)
combine_frames["current_ratio"] = round(combine_frames["current_assets"] / combine_frames["current_liabilities"],2)
keymetrics:pd.DataFrame = combine_frames[["debtToEquity","returnOnEquity","current_ratio"]]
keymetrics


,debtToEquity,returnOnEquity,current_ratio
date,,,
2020,0.05,0.16,3.10
2021,0.04,0.15,2.80
2022,0.00,0.12,3.76
2023,0.00,0.08,3.65
2024,0.00,0.16,3.68


<!-- @title Vigilant leader -->
## Vigilant leader

### Rule 1 - importance of `debtEquityRatio`
A  company can choose to finance its investment using its equity or debt. However, a company would have much better flexibility if it use equity. Warren Buffet like to invest in company whose `debtEquity` ratio is below `0.5`

### Rule 2 - current ratio

**Current Asset** will be converted to cash within the next 12 months such the inventory expected to be sold. On the other hand, **Current Liability** is the money that company will need to pay within next 12 months, so it could be raw material received from the supplier but they have not paid for it. `Current ratio = current asset / current liability`. Really high current ratio can indicate that the company having hardtime collecting money from the vender. If current ratio below 1, the company might have to acquire new debt to pay off the existing debt. In general, current ratio between 1.5 to 2.5 is desireable. A current ratio above 5 might indicate bad money mannagement.

### Rule 3 - Strong and consistent return on equity

ROE shows you how effective the mangement reinvest your profit in the business. `ROE = Net Income / Shareholder's Equity` Remember that `equity = asset - libility`, and all profit made will go under `equity` in the balance sheet, so ROE help us examine portion of equity that is net income. In general, we should look for companies that have a consistent ROE above `8%` over the last ten 10 years. After identifying the trend, we should also compare it with the competitor. When examing ROE, it is very important to compare it side by side with debt to equity ratio. The company can potentially use debt instead of its stockholder's equity to keep ROE consistent or even higher, so we need to be careful of company with `debtEquity ratio` higher than 0.5. Here is an example demonstrating such account tricks

![ROE accounting tricks](https://i.ibb.co/twfwHZg7/roe-accounting-tricks.png)

In the picture above, we see the company using debt instead of shareholder's equity to make ROE look nicer.

### Rule 4 - Appropriate management incentives

If a management is given the incentive to solely focus on the share price, the management is also given the incentive not to pay out dividends, but rather to retain all earnings even though there are no good projects to invest in.
You want a company that disclose how much the base salary is and how much is variable, and which indicators the management is measured on.





In [8]:
#@title table for vigilant leader

keymetrics.style.set_caption(f"Table for {stock_symbol}")

save_dataframe(keymetrics, f'Table 1: Key Metrics for {stock_symbol}', report_path)

C:\Users\fbcha\AppData\Local\Temp\ipykernel_6492\3009390457.py:52: UserWarning: Glyph 21407 (\N{CJK UNIFIED IDEOGRAPH-539F}) missing from font(s) DejaVu Sans.
  pdf.savefig(bbox_inches="tight")
C:\Users\fbcha\AppData\Local\Temp\ipykernel_6492\3009390457.py:52: UserWarning: Glyph 30456 (\N{CJK UNIFIED IDEOGRAPH-76F8}) missing from font(s) DejaVu Sans.
  pdf.savefig(bbox_inches="tight")


figure is saved


In [9]:
# Figure 1: ROE and Debt to Equity Ratio
fig1, roeAxis = plt.subplots()
sns.lineplot(data = keymetrics, x=keymetrics.index, y='returnOnEquity', marker="o", ax=roeAxis, color='blue')
debtToEquityAxis = roeAxis.twinx()
sns.lineplot(data = keymetrics, x=keymetrics.index, y='debtToEquity', marker="o", ax=debtToEquityAxis, color='orange')

debtToEquityAxis.set_ylabel('Debt to Equity Ratio', color='orange')

# Figure 1: ROE axis analysis
roe_mean:float = keymetrics['returnOnEquity'].mean()
roeAxis.axhline(y=roe_mean, color='blue', linestyle='--')
roeAxis.axhline(y=0.08, color='red', linestyle='--')
roeAxis.axhspan(0,0.08, color='red', alpha=0.1)

roeAxis.set_ylim(bottom=0)
roeAxis.set_ylabel('Return on Equity (ROE)', color='blue')
roeAxis.text(keymetrics.index[-1], roe_mean, f' Average ROE: {to_percentage(roe_mean)}', color='blue', va='bottom', ha='left')
roeAxis.text(keymetrics.index[-1], 0.08, ' 8% ROE Threshold', color='red', va='bottom', ha='left')

fig1.legend()
fig1.subplots_adjust(left=0.1, right=0.75, top=0.85, bottom=0.1)
fig1.suptitle(f'Figure 1: ROE and Debt to Equity Ratio for {stock_symbol}')
# fig1_secription:str = (
#     "In Figure 1 above. If ROE is constantly over 8%, the only thing that we need to watch out for is increasing DE ratio. "
#     "If DE ratio is increasing while ROE remain constant, please don't pursue this stock. "
#     "In that case, company might be using the desperate approach to stablize the ROE."
# )
# fig1.text(0.5, -0.2, s=fig1_secription, ha='center', fontsize = "x-large", wrap=True, color = "#696969")
# plt.show()
save_fig(report_path)

C:\Users\fbcha\AppData\Local\Temp\ipykernel_6492\3136310401.py:20: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  fig1.legend()
C:\Users\fbcha\AppData\Local\Temp\ipykernel_6492\3009390457.py:52: UserWarning: Glyph 21407 (\N{CJK UNIFIED IDEOGRAPH-539F}) missing from font(s) DejaVu Sans.
  pdf.savefig(bbox_inches="tight")
C:\Users\fbcha\AppData\Local\Temp\ipykernel_6492\3009390457.py:52: UserWarning: Glyph 30456 (\N{CJK UNIFIED IDEOGRAPH-76F8}) missing from font(s) DejaVu Sans.
  pdf.savefig(bbox_inches="tight")


figure is saved


In Figure 1 above. If ROE is constantly over 8%, the only thing that we need to watch out for is increasing DE ratio. If DE ratio is increasing while ROE remain constant, please don't pursue this stock. In that case, company might be using the desperate approach to stablize the ROE.

In [10]:
# Figure 2: Debt to Equity Ratio 
fig2, debtToEquityAxis = plt.subplots()
sns.lineplot(data = keymetrics, x=keymetrics.index, y='debtToEquity', marker="o", ax=debtToEquityAxis, color='orange', label='Debt to Equity Ratio')
# debtToEquityAxis.set_ylabel('Debt to Equity Ratio', color='orange')
debtToEquity_mean:float = keymetrics['debtToEquity'].mean()
debtToEquityAxis.axhline(y=debtToEquity_mean, color='orange', linestyle='--')

# Figure 2: label the matrics
debtToEquityAxis.axhline(y = debtToEquity_mean, color='orange', linestyle='--')
debtToEquityAxis.axhline(y=0.5, color = 'green', linestyle='--')
debtToEquityAxis.axhspan(0,0.5, color='green', alpha=0.1)
debtToEquityAxis.set_ylim(bottom=0)
debtToEquityAxis.text(keymetrics.index[-1], debtToEquity_mean, f' Average DE Ratio: {debtToEquity_mean:.2f}', color='orange', va='bottom', ha='left')
debtToEquityAxis.text(keymetrics.index[0], 0.5, ' Debt to Equity Ratio Threshold: 0.5', color='green', va='bottom', ha='left')

fig2.subplots_adjust(left=0.1, right=0.75, top=0.85, bottom=0.1)


fig2.suptitle(f'Figure 2: Debt to Equity Ratio for {stock_symbol}')

#
save_fig(report_path)

C:\Users\fbcha\AppData\Local\Temp\ipykernel_6492\3009390457.py:52: UserWarning: Glyph 21407 (\N{CJK UNIFIED IDEOGRAPH-539F}) missing from font(s) DejaVu Sans.
  pdf.savefig(bbox_inches="tight")
C:\Users\fbcha\AppData\Local\Temp\ipykernel_6492\3009390457.py:52: UserWarning: Glyph 30456 (\N{CJK UNIFIED IDEOGRAPH-76F8}) missing from font(s) DejaVu Sans.
  pdf.savefig(bbox_inches="tight")


figure is saved


In figure 2, check if DE ratio is constantly below or around `0.5`. This is very important if we want to hold it for a long period of time.

In [11]:
import math
# Figure 3: Current Ratio plot
fig3, currentRatioAxis = plt.subplots()
fig3.suptitle(f'Figure 3: Current Ratio for {stock_symbol}')

sns.lineplot(data = keymetrics, x=keymetrics.index, y='current_ratio', marker="o", ax=currentRatioAxis, color='purple', label='Current Ratio')
currentRatio_mean:float = keymetrics['current_ratio'].mean()
# Figure 3: label the matrics
currentRatioAxis.axhline(y = currentRatio_mean, color='purple', linestyle='--')
currentRatioAxis.axhline(y=2.5, color = 'green', linestyle='--')
currentRatioAxis.axhline(y=1.5, color = 'green', linestyle='--')
currentRatioAxis.axhline(y=5, color = 'red', linestyle='--')
_,ymax = currentRatioAxis.get_ylim()
currentRatioAxis.axhspan(5,ymax, color='red', alpha=0.1)
currentRatioAxis.axhspan(1.5,2.5, color='green', alpha=0.1)
currentRatioAxis.text(keymetrics.index[-1], currentRatio_mean, f'average current ratio: {currentRatio_mean:.2f}', color='purple', va='bottom', ha='left')
currentRatioAxis.text(keymetrics.index[0], 2.5, s=f'optimal current ratio range 1.5~2.5', color='green', va='bottom', ha='left')
currentRatioAxis.text(keymetrics.index[0], 1.5, s=f'optimal current ratio range 1.5~2.5', color='green', va='top', ha='left')
currentRatioAxis.set_ylim(bottom=0, top= ymax)

# fig3_description:str = (
#     "In Figure 3 above. A current ratio between 1.5 and 2.5 is considered optimal, indicating that the company has a healthy balance between liquidity and efficient use of assets. "
#     "A current ratio below 1.5 may suggest potential liquidity issues, while a ratio above 2.5 could indicate that the company is not utilizing its assets effectively."
#     "As long as the current ratio is below 5 and stays consistent, it is generally acceptable."
# )
# fig3.text(0.5, -0.2, s=fig3_description, ha='center', fontsize = "x-large", wrap=True, color = "#696969")
save_fig(report_path)

C:\Users\fbcha\AppData\Local\Temp\ipykernel_6492\3009390457.py:52: UserWarning: Glyph 21407 (\N{CJK UNIFIED IDEOGRAPH-539F}) missing from font(s) DejaVu Sans.
  pdf.savefig(bbox_inches="tight")
C:\Users\fbcha\AppData\Local\Temp\ipykernel_6492\3009390457.py:52: UserWarning: Glyph 30456 (\N{CJK UNIFIED IDEOGRAPH-76F8}) missing from font(s) DejaVu Sans.
  pdf.savefig(bbox_inches="tight")


figure is saved


In the Figure 3 above, really high current ratio can indicate that the company having hardtime collecting money from the vender. If current ratio below 1, the company might have to acquire new debt to pay off the existing debt. In general, current ratio between 1.5 to 2.5 is desireable. A current ratio above 5 might indicate bad money mannagement.

## Principle 2 - A company must have long-term prospect

### Rule 1 - persistent products
After 30 years, it is unlikely that consumer will not change the way they smartphone today. Warren Buffet use this question to determine whether or not he invest in a company: **"Will the Internet change the way we use the product?"**

### Rule 2 - minimize tax

A country often encourage long-term investment. You would have to pay tax on your capital gain like the ordinary income. If you hold onto an investment for more than one year, the rate is usually fixed.



## Priciple 3 - A company must be stable and understandable

### Rule 1 - stable book value growth from the owner's earning
We should look at the stability and growth of book value and earnings per share.It shows that book value growth comes from earnings. When there is increase in equity, there must be an increase in asset or decrease in liability. Dividend paid can reduce equity.

In general, we will see growth in EPS and book value per share, which would reflect in stable ROE value. Since dividend can reduce equity, we should also try to fit dividend into the trend line.


In [12]:
#@title Principle 3 code
combine_frames["bookvalue_PerShare"] = combine_frames["totalEquity"] / combine_frames["share_outstanding"]
keymetric_frame=combine_frames[["bookvalue_PerShare","eps"]]
keymetric_frame.style.set_caption(f"Table for {stock_symbol}")
print(keymetric_frame)
save_dataframe(keymetric_frame, f'Table 2: Book Value per Share and EPS for {stock_symbol}', report_path)


      bookvalue_PerShare    eps
date                           
2020           65.039792  10.60
2021           73.236956  11.32
2022           64.404332   7.41
2023           69.000636   5.57
2024           79.713872  12.52
figure is saved


C:\Users\fbcha\AppData\Local\Temp\ipykernel_6492\3009390457.py:52: UserWarning: Glyph 21407 (\N{CJK UNIFIED IDEOGRAPH-539F}) missing from font(s) DejaVu Sans.
  pdf.savefig(bbox_inches="tight")
C:\Users\fbcha\AppData\Local\Temp\ipykernel_6492\3009390457.py:52: UserWarning: Glyph 30456 (\N{CJK UNIFIED IDEOGRAPH-76F8}) missing from font(s) DejaVu Sans.
  pdf.savefig(bbox_inches="tight")


In [13]:


plt.fill_between(keymetric_frame.index,keymetric_frame['bookvalue_PerShare'], color='Blue', alpha=0.5,label='Book Value Per Share')

plt.fill_between(keymetric_frame.index,keymetric_frame['eps'], color='lightblue', alpha=0.5,label='Earning Per Share', hatch='//')

plt.legend()
plt.suptitle(f'Figure 4: Book Value Per Share and Net Income Per Share for {stock_symbol}')
save_fig(report_path)

C:\Users\fbcha\AppData\Local\Temp\ipykernel_6492\3009390457.py:52: UserWarning: Glyph 21407 (\N{CJK UNIFIED IDEOGRAPH-539F}) missing from font(s) DejaVu Sans.
  pdf.savefig(bbox_inches="tight")
C:\Users\fbcha\AppData\Local\Temp\ipykernel_6492\3009390457.py:52: UserWarning: Glyph 30456 (\N{CJK UNIFIED IDEOGRAPH-76F8}) missing from font(s) DejaVu Sans.
  pdf.savefig(bbox_inches="tight")


figure is saved


## Priciple 4 -Buy at attractive prices

### Rule 1 - margin of safety

The margin of safety is the difference between the share price and the intrinsic value.

### Rule 2 - Low price-earning ratio

`PE ratio = market price of the company / net income`. Warren Buffet suggests that you buy stock with PE below 15. Estimated earning can dramatically change forward PE ratio, so we should also monitor the stability of earning.

### Rule 3 - low price-to-book ratio

`Price to book ratio = market of the company per share / equity per share`. This ratio measure how much investor pay for every $1 of company's equity. Benjamin Graham would try to find companies that had a P/B ratio below 1.5.

### Rule 4 - set a safe discount rate
By using discount rate, an investor can take a business's estimated future cash flows and discount them back to today's value. If a ten-year bond has a 3% return, the investor should never discount a ten-year investment lower than 3% annually. Discount rate are based on opportunity cost, because there might be other investment opportunities that could produce larger return with less risk.

In [14]:
#@title Income Ratio analysis

income_frames["grossProfit_margin"] = income_frames["gross_profit"] / income_frames["revenue"]
income_frames["operatingIncome_margin"] = income_frames["operating_income"] / income_frames["revenue"]
income_frames["netIncome_margin"] = income_frames["net_income"] / income_frames["revenue"]
income_frames["interestCoverage_ratio"] = income_frames["operating_income"] / income_frames["interest_expense"]

income_frames['grossProfit_margin'] = income_frames["grossProfit_margin"].map(to_percentage)
income_frames['operatingIncome_margin'] = income_frames["operatingIncome_margin"].map(to_percentage)
income_frames['netIncome_margin'] = income_frames["netIncome_margin"].map(to_percentage)
income_frames["change_revenue"] = income_frames["revenue"].pct_change().map(to_percentage)

keymetric_frame=income_frames[["grossProfit_margin","operatingIncome_margin","netIncome_margin","interestCoverage_ratio","change_revenue"]]

keymetric_frame.style.set_caption(f"Table for {stock_symbol}")
print(keymetric_frame)
save_dataframe(keymetric_frame, f'Table 3: Income Ratios for {stock_symbol}', report_path)

     grossProfit_margin operatingIncome_margin netIncome_margin  \
date                                                              
2020            57.91 %                22.69 %          17.88 %   
2021            57.07 %                20.73 %          17.91 %   
2022            53.97 %                 0.17 %          20.11 %   
2023            57.60 %                12.22 %          13.68 %   
2024            62.03 %                21.03 %          21.71 %   
TTM             61.85 %                22.00 %          20.91 %   

      interestCoverage_ratio change_revenue  
date                                         
2020             -332.077047          nan %  
2021             -257.267870         8.01 %  
2022               -1.858333       -40.62 %  
2023             -190.072398        11.84 %  
2024             -372.360152        43.08 %  
TTM              -390.744611         9.34 %  


C:\Users\fbcha\AppData\Local\Temp\ipykernel_6492\3009390457.py:52: UserWarning: Glyph 21407 (\N{CJK UNIFIED IDEOGRAPH-539F}) missing from font(s) DejaVu Sans.
  pdf.savefig(bbox_inches="tight")
C:\Users\fbcha\AppData\Local\Temp\ipykernel_6492\3009390457.py:52: UserWarning: Glyph 30456 (\N{CJK UNIFIED IDEOGRAPH-76F8}) missing from font(s) DejaVu Sans.
  pdf.savefig(bbox_inches="tight")


figure is saved


|                         	| Formula                             	| Note                                                                                                                                                                                                                    	|
|-------------------------	|-------------------------------------	|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------	|
| Gross profit margin     	| gross profit / revenue              	| This ratio tells us how efficient  the company is at controlling their direct cost                                                                                                                                      	|
| Operating income ratio  	| operating income / revenue          	| compared to gross profit margin, this ratio take into the secondary expense such as administration, marketing, distribution                                                                                             	|
| Net income ratio        	| Net income / revenue                	| When looking at this ratio, this is  the money that the investors made  compared to money the business collected for the sale of every product                                                                          	|
| Interest Coverage ratio 	| operating income / interest expense 	| This is very important ratio for minimizing your risk. It shows company's ability to keep its head above water. As a rule of thumb, the ratio need to be consistently above 5 for a company to be considered as stable. 	|


In [15]:
#@title Balance sheet ratio
# balance_columns:list[str]=["fiscalYear","totalCurrentAssets","inventory","totalCurrentLiabilities"]
# balance_frame = pd.DataFrame(data=balance_data,columns=balance_columns)
# balance_frame = balance_frame.set_index(keys=["fiscalYear"],drop=True)
combine_frames["acid test ratio"] = (combine_frames["current_assets"] - combine_frames["inventory"]) / combine_frames["current_liabilities"]
combine_frames["inventory turnover"] = combine_frames['cost_of_goods_sold'] / combine_frames["inventory"]
combine_frames['receivables turnover'] = combine_frames['revenue'] / combine_frames['account_receivable']
combine_frames['payables turnover'] = combine_frames['cost_of_goods_sold'] / combine_frames['account_payable']

keymetric_frame = combine_frames[["acid test ratio","inventory turnover","receivables turnover","payables turnover"]]

keymetric_frame.style.set_caption(f"Table for {stock_symbol}")
print(keymetric_frame)
save_dataframe(keymetric_frame, f'Table 4: Turnover ratios for {stock_symbol}', report_path)
#@title Balance sheet ratio plot

      acid test ratio  inventory turnover  receivables turnover  \
date                                                              
2020         2.779648            4.457249              5.462939   
2021         2.429081            3.610698              8.667407   
2022         3.160945            2.434850              8.203408   
2023         3.278726            3.641967              5.890033   
2024         3.255348            3.339636              7.365292   

      payables turnover  
date                     
2020           1.797007  
2021           1.624128  
2022           1.498572  
2023           1.529222  
2024           1.515460  


C:\Users\fbcha\AppData\Local\Temp\ipykernel_6492\3009390457.py:52: UserWarning: Glyph 21407 (\N{CJK UNIFIED IDEOGRAPH-539F}) missing from font(s) DejaVu Sans.
  pdf.savefig(bbox_inches="tight")
C:\Users\fbcha\AppData\Local\Temp\ipykernel_6492\3009390457.py:52: UserWarning: Glyph 30456 (\N{CJK UNIFIED IDEOGRAPH-76F8}) missing from font(s) DejaVu Sans.
  pdf.savefig(bbox_inches="tight")


figure is saved


|                                             	| Formula                                         	| Note                                                                                                                                                                                                                                                  	|
|---------------------------------------------	|-------------------------------------------------	|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------	|
| Acid test ratio                             	| (current asset - inventory) / current liability 	| Assuming nothing is sold in the inventory, do we still expect to to receive more than we need to pay out during next 12 month.  This is more conservative current ratio.                                                                              	|
| Inventory Turnover ratio (Efficiency ratio) 	| Cost of Revenue / Inventory average                     	| This ratio tells you how many times the inventory has been filled. It is preferable to have inventory filled 4 times and above annually. If you are working with quarterly statement, you need to multiply this number by 4.                          	|
| Receivable Turnover ratio                   	| Turnover (Revenue) / Account Receivable average         	| In general, this ratio is the number of times the cash of sales are collected. 365 days divided by this ratio usually means the number of days to collect the cash after the sales is made.  5 - 7 is optimal, but it really depends on the industry. 	|
| Payable Turnover ratio                      	| Cost of Revenue / Account Payable               	| Take 365 days divided by this ratio, we would get how many days the company pay its supplier after the purchase is made                                                                                                                               	|

In [16]:
# figure for acid test ratio
fig4, acidtestAxis = plt.subplots()
sns.lineplot(data = keymetric_frame, x=keymetric_frame.index, y='acid test ratio', marker="o", ax=acidtestAxis, color='brown', label='Acid Test Ratio')
acidtest_mean:float = keymetric_frame['acid test ratio'].mean()
acidtestAxis.axhline(y=acidtest_mean, color='blue', linestyle='--')

acidtestAxis.axhline(y=1, color = 'red', linestyle='--')
acidtestAxis.axhspan(0,1, color='red', alpha=0.1)
acidtestAxis.set_ylim(bottom=0)
acidtestAxis.text(keymetric_frame.index[-1], acidtest_mean, f' Average Acid Test Ratio: {acidtest_mean:.2f}', color='blue', va='bottom', ha='left')
acidtestAxis.text(keymetric_frame.index[0], 1, ' Acid Test Ratio Threshold: 1', color='red', va='top', ha='left')

fig4.subplots_adjust(left=0.1, right=0.75, top=0.85, bottom=0.1)
fig4.suptitle(f'Figure 5: Acid Test Ratio for {stock_symbol}')

save_fig(report_path)

C:\Users\fbcha\AppData\Local\Temp\ipykernel_6492\3009390457.py:52: UserWarning: Glyph 21407 (\N{CJK UNIFIED IDEOGRAPH-539F}) missing from font(s) DejaVu Sans.
  pdf.savefig(bbox_inches="tight")
C:\Users\fbcha\AppData\Local\Temp\ipykernel_6492\3009390457.py:52: UserWarning: Glyph 30456 (\N{CJK UNIFIED IDEOGRAPH-76F8}) missing from font(s) DejaVu Sans.
  pdf.savefig(bbox_inches="tight")


figure is saved


In [17]:
fig5, inventoryTurnoverAxis = plt.subplots()
sns.lineplot(data = keymetric_frame, x=keymetric_frame.index, y='inventory turnover', marker="o", ax=inventoryTurnoverAxis, color='green', label='Inventory Turnover')
inventoryTurnover_mean:float = keymetric_frame['inventory turnover'].mean()
inventoryTurnoverAxis.axhline(y=inventoryTurnover_mean, color='orange', linestyle='--')  

inventoryTurnoverAxis.axhline(y= 4, color='red', linestyle='--')
inventoryTurnoverAxis.axhspan(0,4, color='red', alpha=0.1)
inventoryTurnoverAxis.set_ylim(bottom=0)
inventoryTurnoverAxis.text(keymetric_frame.index[-1], inventoryTurnover_mean, f' Average Inventory Turnover: {inventoryTurnover_mean:.2f}', color='orange', va='bottom', ha='left')
inventoryTurnoverAxis.text(keymetric_frame.index[0], 4, ' Inventory Turnover Threshold: 4', color='red', va='top', ha='left')
fig5.subplots_adjust(left=0.1, right=0.75, top=0.85, bottom=0.1)
fig5.suptitle(f'Figure 5: Inventory Turnover for {stock_symbol}')

save_fig(report_path)


C:\Users\fbcha\AppData\Local\Temp\ipykernel_6492\3009390457.py:52: UserWarning: Glyph 21407 (\N{CJK UNIFIED IDEOGRAPH-539F}) missing from font(s) DejaVu Sans.
  pdf.savefig(bbox_inches="tight")
C:\Users\fbcha\AppData\Local\Temp\ipykernel_6492\3009390457.py:52: UserWarning: Glyph 30456 (\N{CJK UNIFIED IDEOGRAPH-76F8}) missing from font(s) DejaVu Sans.
  pdf.savefig(bbox_inches="tight")


figure is saved


In [18]:
# cashflow ratio

combine_frames["free cash flow to revenue"] = combine_frames["free_cashflow"] / combine_frames["revenue"]
combine_frames["Investing cash flow to operating cash flow"] = -combine_frames["investing_cash_flow"] / combine_frames["operating_cash_flow"]
combine_frames["free_cashflow"] = combine_frames["free_cashflow"].map(lambda x:f"{x:.2e}")
keymetric_frame=combine_frames[["free_cashflow","free cash flow to revenue","Investing cash flow to operating cash flow"]]
keymetric_frame.style.set_caption(f"Table 6: cashflow for {stock_symbol}")

print(keymetric_frame)

save_dataframe(keymetric_frame, f'Table 6: Cashflow Ratios for {stock_symbol}', report_path)


     free_cashflow  free cash flow to revenue  \
date                                            
2020     -1.27e+09                  -0.156201   
2021      1.59e+09                   0.180461   
2022      6.28e+07                   0.012019   
2023      1.10e+09                   0.187477   
2024      1.47e+09                   0.175820   

      Investing cash flow to operating cash flow  
date                                              
2020                                    1.080097  
2021                                    0.425292  
2022                                   -1.126180  
2023                                    0.151104  
2024                                    0.410135  
figure is saved


C:\Users\fbcha\AppData\Local\Temp\ipykernel_6492\3009390457.py:52: UserWarning: Glyph 21407 (\N{CJK UNIFIED IDEOGRAPH-539F}) missing from font(s) DejaVu Sans.
  pdf.savefig(bbox_inches="tight")
C:\Users\fbcha\AppData\Local\Temp\ipykernel_6492\3009390457.py:52: UserWarning: Glyph 30456 (\N{CJK UNIFIED IDEOGRAPH-76F8}) missing from font(s) DejaVu Sans.
  pdf.savefig(bbox_inches="tight")


In [19]:
fig6, freeCashflowAxis = plt.subplots()

investingCashflowAxis = freeCashflowAxis.twinx()

sns.lineplot(data = keymetric_frame, x=keymetric_frame.index, y='free cash flow to revenue', marker="o", ax=freeCashflowAxis, color='magenta', label='Free Cash Flow to Revenue')

sns.lineplot(data = keymetric_frame, x=keymetric_frame.index, y='Investing cash flow to operating cash flow', marker="o", ax=investingCashflowAxis, color='cyan', label='Investing Cash Flow to Operating Cash Flow')
fcf_mean:float = keymetric_frame['free cash flow to revenue'].mean()


freeCashflowAxis.axhline(y=fcf_mean, color='magenta', linestyle ='--')
freeCashflowAxis.axhline(y=0.05, color='red', linestyle ='--')
freeCashflowAxis.axhspan(0,0.05, color='red', alpha=0.1)

freeCashflowAxis.set_ylim(bottom=0)
freeCashflowAxis.set_ylabel('Free Cash Flow to Revenue', color='magenta')
freeCashflowAxis.text(keymetric_frame.index[-1], fcf_mean, f' Average Free Cash Flow to Revenue: {to_percentage(fcf_mean)}', color='magenta', va='bottom', ha='left')
freeCashflowAxis.text(keymetric_frame.index[-1], 0.05, ' 5% Free Cash Flow to Revenue Threshold', color='red', va='bottom', ha='left')

fig6.legend()
fig6.subplots_adjust(left=0.1, right=0.75, top=0.85, bottom=0.1)
fig6.suptitle(f'Figure 6: Free Cash Flow to Revenue for {stock_symbol}')
save_fig(report_path)

C:\Users\fbcha\AppData\Local\Temp\ipykernel_6492\3009390457.py:52: UserWarning: Glyph 21407 (\N{CJK UNIFIED IDEOGRAPH-539F}) missing from font(s) DejaVu Sans.
  pdf.savefig(bbox_inches="tight")
C:\Users\fbcha\AppData\Local\Temp\ipykernel_6492\3009390457.py:52: UserWarning: Glyph 30456 (\N{CJK UNIFIED IDEOGRAPH-76F8}) missing from font(s) DejaVu Sans.
  pdf.savefig(bbox_inches="tight")


figure is saved


|                                                 	| Formula                                        	| Note                                                                                                                                                                                                                                         	|
|-------------------------------------------------	|------------------------------------------------	|----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------	|
| Free Cash Flow                                  	| operating cash flow - Property, Equipment, net 	|                                                                                                                                                                                                                                              	|
| FCF to revenue ratio                            	| free cash flow / revenue                       	| This ratio strip away the sales on credit and measure how much cash goes from sales to owner. It is optimal to see at least 5%  over the years                                                                                               	|
| Investing cash flow to operating cashflow ratio 	| Investing cash flow / operating cashflow       	| This ratio measures how much cash goes from daily operation to reinvestment of the company. Assuming Pepsi has 60% and Coke has 50%, we would  find Coke more attractive, because less shareholder's  money is tied up to grow the business  	|